In [1]:
from mne.io import read_raw_eeglab

from pycrostates.cluster import ModKMeans
from pycrostates.datasets import lemon


# load sample dataset
raw_fname = lemon.data_path(subject_id='010017', condition='EC')
raw = read_raw_eeglab(raw_fname, preload=True)
raw.crop(0, 10)  # crop the dataset to speed up computation
raw.pick('eeg')  # select EEG channels
raw.set_eeg_reference('average')  # Apply a common average reference

Reading C:\Users\victor.ferat\pycrostates_data\PREPROCESSED_LEMON\sub-010017_EC.fdt
Reading 0 ... 119451  =      0.000 ...   477.804 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


C:\Users\victor.ferat\AppData\Local\Temp\ipykernel_320\2156721480.py:9: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = read_raw_eeglab(raw_fname, preload=True)
C:\Users\victor.ferat\AppData\Local\Temp\ipykernel_320\2156721480.py:9: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = read_raw_eeglab(raw_fname, preload=True)


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,64 points
Good channels,61 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,0.00 Hz
Lowpass,125.00 Hz


In [ ]:
n_clusters = 5
ModK = ModKMeans(n_clusters=n_clusters, random_state=42)

In [ ]:
ModK.fit(raw, n_jobs=5)

In [3]:
import numpy as np

data = raw.get_data()
std = np.std(data, axis=1)
rms = np.sqrt(np.mean(data**2, axis=1))
np.allclose(std,  rms)

In [17]:
data = raw.get_data() + 10
std = np.std(data, axis=1)
rms = np.sqrt(np.mean(data**2, axis=1))
np.allclose(std,  rms)

False

In [15]:
from pycrostates.utils import _corr_vectors

a = _corr_vectors(data[0], data[1])
print(a)
a = _corr_vectors(data[0]+10, data[1])
print(a)

0.9547847939902916
0.9547847939903947


In [14]:
def cosine_similarity(vector1, vector2):
    # Normalize the vectors
    normalized_vector1 = vector1 / np.linalg.norm(vector1)
    normalized_vector2 = vector2 / np.linalg.norm(vector2)
    
    # Compute dot product
    dot_product = np.dot(normalized_vector1, normalized_vector2)
    
    return dot_product

a = cosine_similarity(data[0], data[1])
print(a)
a = cosine_similarity(data[0]+10, data[1])
print(a)

0.9547932716947615
-0.013669674931094528


In [13]:
def R(x, y):
    numerator = np.dot(x.T, y)
    denominator = np.sqrt(np.dot(x.T, x)) * np.sqrt(np.dot(y.T, y))
    return np.cos(np.arccos(np.clip(numerator / denominator, -1.0, 1.0)))

a = R(data[0], data[1])
print(a)
a = R(data[0]+10, data[1])
print(a)

0.9547932716947615
-0.013669674931094495


In [16]:

def _cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    return dot_product

a = _cosine_similarity(data[0], data[1])
print(a)
a = _cosine_similarity(data[0]+10, data[1])
print(a)

0.9547932716947615
-0.013669674931094522
